In [274]:
import numpy as np
import matplotlib.pyplot as plt
import requests as req
import json
import os
import pandas as pd
import re

In [275]:
#mining
version = '2.7'
access_token = "EAACEdEose0cBAPbQm9ZCMsA9X2aFEKWcH7368TI8XCBlcCVwpZAtkznBkT3sLhgFihDptvZBuGvpRBtK9KmZCR5AarFGWWQzrQTS31nwUkPOoNboiGwtZAZAL3ITVro6ZCledlfd9f68zZAyk2OGZCzDNAUK30aKJrh828bahdEA9rAZDZD"

In [385]:
target = "cefismausp"

In [386]:
url = "https://graph.facebook.com/v%s/" % version
url += "%s/feed/" % target
i = 0
p = {'access_token': access_token}
r = req.get(url, params=p)
data = r.json()
filepath = "%s-%s" % (target, i)
with open(filepath, 'w') as fil:
    json.dump(data, fil)
i += 1
paging_token = data['paging']['next']

In [387]:
while 1 == 1:
    r = req.get(data['paging']['next'])
    data = r.json()
    filepath = "%s-%s" % (target, i)
    with open(filepath, 'w') as fil:
        json.dump(data, fil)
    i += 1
    paging_token = data['paging']['next']

KeyError: 'paging'

In [388]:
def validate(var, keys, dat):
    if var in keys:
        return dat[var]
    else:
        return None

def validate_2(keys, dat):
    id = validate('id', keys, dat)
    created_time = validate('created_time', keys, dat)
    message = validate('message', keys, dat)
    story = validate('story', keys, dat)
    return (id, created_time, message, story)

In [389]:
#processing
id = []
created_time = []
message = []
story = []
todos = []
todos.append(("id", "created_time", "message", "story"))
for filename in os.listdir("cefisma"):
    if filename[-4] != "ipynb":
        with open("cefisma/%s" % filename, "r") as fil:
            j = json.loads(fil.read())
            for post in j['data']:
                keys = post.keys()
                todos.append(validate_2(keys, post))              
                

In [390]:
a = pd.DataFrame(todos)
a.to_csv("%s-output.csv" % target, index_label=False)
a = pd.read_csv("%s-output.csv" % target, header=1)

In [393]:
words = []
word_c = []

for message in a.message:
    message = str(message).lower()
    splited = re.findall("\w+", message)
    for word in splited:
        if word in words:
            word_c[words.index(word)] += 1
        else:
            words.append(word)
            word_c.append(1)

In [394]:
res = [words, word_c]
b = pd.DataFrame(list(zip(*res)), columns=["word", "word_count"])



b = b[b.word.str.len() > 3]
b = b.sort_values(['word_count'], ascending=False)
b.to_csv("hehe.csv")
nword_l = np.array(b.word.str.len())
nword_c = np.array(b.word_count)
avg = np.sum(nword_l * nword_c) / np.sum(nword_c)
avg

7.0601492755744184